In [119]:
from ortools.graph.python import max_flow
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
import time
import random

SETUP 

In [ ]:
def input_data():
    with open('/workspaces/rtewr/Project/input.txt', 'r') as f:
        num_papers,num_reviewers,reviews_per_paper = map(int, f.readline().strip().split())
        willing_reviewers = {}
        for i in range(num_papers):
            line = list(map(int, f.readline().strip().split()))
            paper_id = i+1
            reviewers = line[1:]
            willing_reviewers[paper_id] = reviewers
    return num_papers, num_reviewers, reviews_per_paper, willing_reviewers
def reverse_dict(willing_reviewers):
    willing_papers = {}
    for paper, reviewers in willing_reviewers.items():
        for reviewer in reviewers:
            if reviewer not in willing_papers:
                willing_papers[reviewer] = []
            willing_papers[reviewer].append(paper)
    return willing_papers

# Paper Reviewer Assignment: Constraint Programming Model with OR-Tools

## Problem Modeling

### 1. **Variables**
- **Binary Assignment Variables**:  
  `x[(paper, reviewer)]`: Binary variable where:
  - `1` if reviewer `reviewer` is assigned to paper `paper`
  - `0` otherwise  
  **Domain**: Defined only for valid pairs `(paper, reviewer)` where `reviewer ∈ L(paper)`.

- **Load Variables**:  
  `loads[reviewer]`: Integer variable representing the total number of papers assigned to reviewer `reviewer`.  
  **Domain**: `0 ≤ loads[reviewer] ≤ num_papers`.

- **Max Load Variable**:  
  `max_load`: Integer variable representing the maximum load across all reviewers (objective to minimize).  
  **Domain**: `0 ≤ max_load ≤ num_papers`.

---


### 2. **Constraints**
1. **Paper Assignment Constraint**:  
   Each paper must be reviewed by exactly `reviews_per_paper` reviewers:  
   ```python
   for paper in range(1, num_papers + 1):
       model.Add(sum(x[(paper, reviewer)] for reviewer in willing_reviewers[paper]) == reviews_per_paper)


In [ ]:
@time_execution
def main()->None:
    # Read input data
    num_papers, num_reviewers, reviews_per_paper, willing_reviewers = input_data()  
    # Create the model
    model = cp_model.CpModel()
    # Create binary variables for each paper-reviewer pair
    x= {}
    for paper in range(1, num_papers + 1):
        for reviewer in willing_reviewers[paper]:
            x[(paper, reviewer)] = model.NewBoolVar(f'x[{paper},{reviewer}]')

    # Each paper must be reviewed by exactly reviews_per_paper reviewers
    for paper in range(1, num_papers + 1):
        model.Add(sum(x[(paper, reviewer)] for reviewer in willing_reviewers[paper]) == reviews_per_paper)
    # Load for each reviewer
    loads = {}
    for reviewer in range(1, num_reviewers + 1):
        loads[reviewer] = model.NewIntVar(0, num_papers, f'load[{reviewer}]')
        model.Add(loads[reviewer] == sum(x[(paper, reviewer)] for paper in range(1, num_papers + 1) if (paper, reviewer) in x))
    
    # Add constraints max of loads is minium
    max_load = model.NewIntVar(0, num_papers, 'max_load')
    for reviewer in range(1, num_reviewers + 1):
        model.Add(loads[reviewer] <= max_load)

    # Objective: minimize the maximum load
    model.Minimize(max_load)

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    # Print the solution
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        """print(num_papers)
        for paper in range(1, num_papers + 1):
            print(reviews_per_paper, end=' ')
            for reviewer in willing_reviewers[paper]:
                if solver.Value(x[(paper, reviewer)]) == 1:
                    print(reviewer, end=' ')
            print()"""
        print(solver.ObjectiveValue())
    else:
        print('No solution found.')
    """# Print statistics
    print()
    print('Statistics:')
    print(f'  status   : {solver.StatusName(status)}')
    print(f'  conflicts: {solver.NumConflicts()}')
    print(f'  branches : {solver.NumBranches()}')
    print(f'  wall time: {solver.WallTime()} ms')
    print(f'  load     : {solver.ObjectiveValue()}')"""
if __name__ == "__main__":
    main()


30.0
Execution time: 1.0602 seconds


In [122]:
def pre_processing_data(num_papers,num_reviewers,reviews_per_paper ,willing_reviewers,max_load):
    
    source = 0
    sink = num_papers + num_reviewers + 1

    start_nodes = []
    end_nodes = []
    capacities = []

    # Arcs from source to papers
    start_nodes += [source] * num_papers
    end_nodes += [i for i in range(1, num_papers + 1)]
    capacities += [reviews_per_paper] * num_papers

    # Arcs from papers to reviewers (with correct offset)
    for paper in range(1, num_papers + 1):
        for reviewer in willing_reviewers[paper]:
            start_nodes.append(paper)
            end_nodes.append(reviewer + num_papers)  # Add offset here
            capacities.append(1)

    # Arcs from reviewers to sink
    for reviewer in range(1, num_reviewers + 1):
        start_nodes.append(reviewer + num_papers)
        end_nodes.append(sink)
        capacities.append(max_load)
    return start_nodes, end_nodes, capacities

![alt text](image.png)

In [123]:
@time_execution
def max_flow_method(): 
    # Instantiate a SimpleMaxFlow solver.
    smf = max_flow.SimpleMaxFlow()

    # Read input data
    num_papers, num_reviewers, reviews_per_paper, willing_reviewers = input_data()

    #Minimum capactices of max_load
    if (num_papers*reviews_per_paper) % num_reviewers == 0:
        low=(num_papers*reviews_per_paper) // num_reviewers
    else:
        low=(num_papers*reviews_per_paper) // num_reviewers + 1
    
    willing_papers = reverse_dict(willing_reviewers)
    willing_papers=dict(sorted(willing_papers.items(), key=lambda item: len(item[1])))
    high= max(len(papers) for papers in willing_papers.values()) if willing_papers else 0
        
    #Dirichlet's theorem
    max_load= low

    while max_load<=high:        
        start_nodes, end_nodes, capacities = pre_processing_data(num_papers,num_reviewers,reviews_per_paper ,willing_reviewers,max_load)
        #   note: we could have used add_arc_with_capacity(start, end, capacity)
        all_arcs = smf.add_arcs_with_capacity(start_nodes, end_nodes, capacities)

        # Find the maximum flow between node 0 and node 4.
        status = smf.solve(0, num_papers + num_reviewers + 1)

        if (status == smf.OPTIMAL) and smf.optimal_flow()== num_papers * reviews_per_paper:
            # Print the solution
            """print(num_papers)
            solution_flows = smf.flows(all_arcs)
            arc_indices = {arc: i for i, arc in enumerate(zip(start_nodes, end_nodes))}
       
            for paper in range(1, num_papers + 1):
                print(reviews_per_paper, end=' ')
                assigned_reviewers = []
                
                # Check all arcs from this paper to reviewers
                for reviewer in willing_reviewers[paper]:
                    arc = (paper, reviewer + num_papers)
                    if arc in arc_indices:
                        flow_index = arc_indices[arc]
                        if solution_flows[flow_index] == 1:
                            assigned_reviewers.append(reviewer)
                
                # Print assigned reviewers
                for rev in assigned_reviewers[:reviews_per_paper]:  # Ensure we don't exceed required reviews
                    print(rev, end=' ')
                print()"""
            print(max_load) 
            break
        else:
            max_load += 1

       
        
max_flow_method()

30
Execution time: 0.0061 seconds


In [124]:
def matching_papers(num_papers, num_reviewers, reviews_per_paper, willing_reviewers):
    load = [0] * (num_reviewers + 1)
    sorted_dict = dict(sorted(willing_reviewers.items(), key=lambda item: len(item[1])))
    selected_reviewers = {}
    for paper,reviewers in sorted_dict.items():
        #Sort the reviewers by their current load
        reviewers.sort(key=lambda x: load[x])
        # Select the first K reviewers
        selected_reviewers[paper] = reviewers[:reviews_per_paper]
        # Update the load of the selected reviewers
        for reviewer in selected_reviewers[paper]:
            load[reviewer] += 1
    # Find the maximum load
    max_load = max(load[1:])
    return max_load, selected_reviewers
@time_execution
def greedy_method(): 
    # Read input data
    num_papers, num_reviewers, reviews_per_paper, willing_reviewers = input_data()
    
    # Call the matching function
    max_load, selected_reviewers = matching_papers(num_papers, num_reviewers, reviews_per_paper, willing_reviewers)
    
    # Print the result
    """print(num_papers)
    for paper in range(1, num_papers + 1):
        print(reviews_per_paper, end=' ')
        for reviewer in selected_reviewers.get(paper, []):
            print(reviewer, end=' ')
        print()"""
    print(max_load)
    
greedy_method() 

31
Execution time: 0.0050 seconds


In [125]:
def matching_papers_2(num_papers, num_reviewers, reviews_per_paper, willing_reviewers):
    load = {i: 0 for i in range(1, num_reviewers + 1)}  # Fixed: Start from 1 instead of 0
    sorted_dict = dict(sorted(willing_reviewers.items(), key=lambda item: len(item[1])))
    selected_reviewers = {}
    for paper,reviewers in sorted_dict.items():
        #Sort the reviewers by their current load
        reviewers.sort(key=lambda x: load[x])
        # Select the first K reviewers
        selected_reviewers[paper] = reviewers[:reviews_per_paper]
        # Update the load of the selected reviewers
        for reviewer in selected_reviewers[paper]:
            load[reviewer] += 1
    # Find the maximum load
    max_load = max(load.values())
    return max_load, selected_reviewers, load

def local_search(num_papers, num_reviewers, reviews_per_paper, willing_papers, assigned_papers, current_load, depth=0):
    if depth > 100:  # Limit recursion depth
        return assigned_papers, current_load
    
    changed = False
    sorted_load = dict(sorted(current_load.items(), key=lambda item: item[1]))
    #get the max load
    reviewer_of_max_load = max(current_load, key=current_load.get)
    max_load = current_load[reviewer_of_max_load]
    
    for candidate in sorted_load.keys():
        if sorted_load[candidate] < max_load-1:
            # Ensure both reviewers exist in the dictionaries
            if candidate not in willing_papers:
                willing_papers[candidate] = []
            if reviewer_of_max_load not in assigned_papers:
                continue
                
            available_papers = list(set(willing_papers.get(candidate, [])) & set(assigned_papers[reviewer_of_max_load]))
            if len(available_papers) > 0:
                random_paper = random.choice(available_papers)
                assigned_papers[candidate].append(random_paper)
                assigned_papers[reviewer_of_max_load].remove(random_paper)
                # Update the load
                current_load[candidate] += 1
                current_load[reviewer_of_max_load] -= 1
                changed=True
                break
                
    if changed:
        return local_search(num_papers, num_reviewers, reviews_per_paper, willing_papers, assigned_papers, current_load, depth+1)

    return assigned_papers, current_load
@time_execution
def local_search_method():
    num_papers, num_reviewers, reviews_per_paper, willing_reviewers = input_data()
    willing_papers = reverse_dict(willing_reviewers)

    
    max_load, selected_reviewers, current_load = matching_papers_2(num_papers, num_reviewers, reviews_per_paper, willing_reviewers)
    
    # Calculate assigned papers for each reviewer
    assigned_papers = reverse_dict(selected_reviewers)
    
    # Update willing papers by removing already assigned papers
    reviewer_willing_papers = {}
    for i in range(1, num_reviewers+1):
        if i in willing_papers:
            reviewer_willing_papers[i] = list(set(willing_papers[i]) - set(assigned_papers.get(i, [])))
    
    # Perform local search
    optimized_assignments, final_load = local_search(num_papers, num_reviewers, reviews_per_paper, reviewer_willing_papers, assigned_papers, current_load)
    selected_reviewers=reverse_dict(optimized_assignments)
    """print(num_papers)
    for paper, reviewers in selected_reviewers.items():
        print(f"{reviews_per_paper} {' '.join(map(str, reviewers))}")"""
    print(f"Max load: {max(final_load.values())}")

local_search_method()

Max load: 30
Execution time: 0.0071 seconds
